In [ ]:
from RANGE_py.ga_abc import GA_ABC
from RANGE_py.cluster_model import cluster_model
from RANGE_py.energy_calculation import energy_computation

import numpy as np
#import matplotlib.pyplot as plt
import os

from ase import Atoms
from ase.constraints import FixAtoms
from ase.visualize import view
from ase.db import connect
from ase.io import read, write

# Environment setup
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [ ]:
def remove_db(db_name):
    if os.path.exists(db_name):
        os.remove(db_name)

output_db_name = 'structure_pool.db'

# Example 1: at_position + inbox with outside box option 

In [ ]:
copper_13 = 'xyz_structures/Cu13.xyz'
methane = 'xyz_structures/methane.xyz'

input_molecules = [copper_13, methane]
input_num_of_molecules = [1, 5]

input_constraint_type = [ 'at_position', 'in_box' ]
input_constraint_value = [ (5,5,5,0,0,0), (0,0,0,10,10,10, 3,3,3,7,7,7) ]

# Setting cluster" 
cluster = cluster_model(input_molecules, input_num_of_molecules, input_constraint_type, input_constraint_value)
cluster.init_molelcules()
cluster_template, cluster_boundary, cluster_conversion_rule = cluster.generate_bounds()  # Generate modeling setting

# Setting calculator for coarse opt
coarse_opt_parameter = dict(coarse_calc_eps='UFF', coarse_calc_sig='UFF', coarse_calc_chg=0, 
                            coarse_calc_step=10, coarse_calc_fmax=10, coarse_calc_constraint=None)
computation = energy_computation(templates = cluster_template, go_conversion_rule = cluster_conversion_rule,   
                                 calculator = None, geo_opt_para = None, calculator_type = 'structural',    
                                 if_coarse_calc = False, coarse_calc_para = coarse_opt_parameter,)
# Setting the main opt process
remove_db(output_db_name)
optimization = GA_ABC(computation.obj_func_compute_energy, cluster_boundary, colony_size=10,  
                      max_iteration=0, initial_population_scaler=1, output_database = output_db_name )
optimization.run(print_interval=1)  # Start running. Print information with given frequency. 


In [ ]:
db = connect('structure_pool.db')
atoms = db.get_atoms(id=3) 
view( atoms, viewer='x3d' )


In [ ]:
# With built-in coarse-optimization force field 
coarse_opt_parameter = dict(coarse_calc_eps='UFF', coarse_calc_sig='UFF', coarse_calc_chg=0, 
                            coarse_calc_step=20, coarse_calc_fmax=10, coarse_calc_constraint=None)
computation = energy_computation(templates = cluster_template, go_conversion_rule = cluster_conversion_rule,   
                                 calculator = None, geo_opt_para = None, calculator_type = 'structural',    
                                 if_coarse_calc = True, coarse_calc_para = coarse_opt_parameter,)
# Setting the main opt process
remove_db(output_db_name)
optimization = GA_ABC(computation.obj_func_compute_energy, cluster_boundary, colony_size=10,  
                      max_iteration=0, initial_population_scaler=1, output_database = output_db_name )
optimization.run(print_interval=1)  # Start running. Print information with given frequency. 

In [ ]:
db = connect('structure_pool.db')
atoms = db.get_atoms(id=3) 
view( atoms, viewer='x3d' )


# Example 2: at_position + in_sphere_shell

In [ ]:
copper_13 = 'xyz_structures/Cu13.xyz'
water = 'xyz_structures/Water.xyz'

input_molecules = [copper_13, water]
input_num_of_molecules = [1, 20]

input_constraint_type = [ 'at_position', 'in_sphere_shell' ]
input_constraint_value = [ (5,5,5), (5,5,5, 10,10,10, 0.5) ]

# Setting cluster" 
cluster = cluster_model(input_molecules, input_num_of_molecules, input_constraint_type, input_constraint_value)
cluster.init_molelcules()
cluster_template, cluster_boundary, cluster_conversion_rule = cluster.generate_bounds()  # Generate modeling setting

# With built-in coarse-optimization force field 
coarse_opt_parameter = dict(coarse_calc_eps='UFF', coarse_calc_sig='UFF', coarse_calc_chg=0, 
                            coarse_calc_step=20, coarse_calc_fmax=10, coarse_calc_constraint=None)
computation = energy_computation(templates = cluster_template, go_conversion_rule = cluster_conversion_rule,   
                                 calculator = None, geo_opt_para = None, calculator_type = 'structural',    
                                 if_coarse_calc = True, coarse_calc_para = coarse_opt_parameter,)
# Setting the main opt process
remove_db(output_db_name)
optimization = GA_ABC(computation.obj_func_compute_energy, cluster_boundary, colony_size=10,  
                      max_iteration=0, initial_population_scaler=1, output_database = output_db_name )
optimization.run(print_interval=1)  # Start running. Print information with given frequency. 


In [ ]:
db = connect('structure_pool.db')
atoms = db.get_atoms(id=3) 
view( atoms, viewer='x3d' )


# Example 3: at_position + on_surface

In [ ]:
copper_13 = 'xyz_structures/Cu13.xyz'
water = 'xyz_structures/Water.xyz'

input_molecules = [copper_13, water]
input_num_of_molecules = [1, 5]

input_constraint_type = [ 'at_position', 'on_surface' ]
input_constraint_value = [ (5,5,5,0,0,0), (0, (1.9, 2.1), 0,1) ]

# Setting cluster" 
cluster = cluster_model(input_molecules, input_num_of_molecules, input_constraint_type, input_constraint_value)
cluster.init_molelcules()
cluster_template, cluster_boundary, cluster_conversion_rule = cluster.generate_bounds()  # Generate modeling setting

# With built-in coarse-optimization force field 
coarse_opt_parameter = dict(coarse_calc_eps='UFF', coarse_calc_sig='UFF', coarse_calc_chg=0, 
                            coarse_calc_step=20, coarse_calc_fmax=10, coarse_calc_constraint=None)
computation = energy_computation(templates = cluster_template, go_conversion_rule = cluster_conversion_rule,   
                                 calculator = None, geo_opt_para = None, calculator_type = 'structural',    
                                 if_coarse_calc = True, coarse_calc_para = coarse_opt_parameter,)
# Setting the main opt process
remove_db(output_db_name)
optimization = GA_ABC(computation.obj_func_compute_energy, cluster_boundary, colony_size=10,  
                      max_iteration=0, initial_population_scaler=1, output_database = output_db_name )
optimization.run(print_interval=1)  # Start running. Print information with given frequency. 


In [ ]:
db = connect('structure_pool.db')
atoms = db.get_atoms(id=3) 
view( atoms, viewer='x3d' )


# Example 4: at_position + in_pore

In [ ]:
zeolite = 'xyz_structures/Zeolite.xyz'
water = 'xyz_structures/Water.xyz'

input_molecules = [zeolite, water]
input_num_of_molecules = [1, 5]

input_constraint_type = [ 'at_position', 'in_pore' ]
input_constraint_value = [ (), (0, (287,95,96,97,113,106,250,265,270,136,262,258,261), 0.2) ]

# Setting cluster" 
cluster = cluster_model(input_molecules, input_num_of_molecules, input_constraint_type, input_constraint_value,
                       pbc_box=( 20.022, 19.899, 13.383 ) ) # PBC system
cluster.init_molelcules()
cluster_template, cluster_boundary, cluster_conversion_rule = cluster.generate_bounds()  # Generate modeling setting

# With built-in coarse-optimization force field 
coarse_opt_parameter = dict(coarse_calc_eps='UFF', coarse_calc_sig='UFF', coarse_calc_chg=0, 
                            coarse_calc_step=20, coarse_calc_fmax=10, coarse_calc_constraint=None)
computation = energy_computation(templates = cluster_template, go_conversion_rule = cluster_conversion_rule,   
                                 calculator = None, geo_opt_para = None, calculator_type = 'structural',    
                                 if_coarse_calc = True, coarse_calc_para = coarse_opt_parameter,)
# Setting the main opt process
remove_db(output_db_name)
optimization = GA_ABC(computation.obj_func_compute_energy, cluster_boundary, colony_size=10,  
                      max_iteration=0, initial_population_scaler=1, output_database = output_db_name )
optimization.run(print_interval=1)  # Start running. Print information with given frequency. 


In [ ]:
db = connect('structure_pool.db')
atoms = db.get_atoms(id= 3 ) 
view( atoms, viewer='x3d' )


# Example 5: layer and micelle

In [ ]:
substrate = 'xyz_structures/BaTiO3-cub-7layer-TiO.xyz'
water = 'xyz_structures/Water.xyz'

input_molecules = [substrate, water]
input_num_of_molecules = [1, 5]

input_constraint_type = [ 'at_position', 'layer' ]
input_constraint_value = [ (), ( (4,4,18), (14,4,18), (4,14,18), (0,0,30), 0.5, 0,1 ) ]

# Setting cluster" 
cluster = cluster_model(input_molecules, input_num_of_molecules, input_constraint_type, input_constraint_value,
                       pbc_box=(20.26028, 20.26028, 32.13928) ) # PBC system
cluster.init_molelcules()
cluster_template, cluster_boundary, cluster_conversion_rule = cluster.generate_bounds()  # Generate modeling setting

# Constraint
cluster_atoms = Atoms()
for at in cluster_template:
    cluster_atoms += at
ase_constraint = FixAtoms(indices=[at.index for at in cluster_atoms if at.symbol=='Ba' or at.symbol=='Ti' ])

# With built-in coarse-optimization force field 
coarse_opt_parameter = dict(coarse_calc_eps='UFF', coarse_calc_sig='UFF', coarse_calc_chg=0, 
                            coarse_calc_step=20, coarse_calc_fmax=10, coarse_calc_constraint=ase_constraint)
computation = energy_computation(templates = cluster_template, go_conversion_rule = cluster_conversion_rule,   
                                 calculator = None, geo_opt_para = None, calculator_type = 'structural',    
                                 if_coarse_calc = True, coarse_calc_para = coarse_opt_parameter,)
# Setting the main opt process
remove_db(output_db_name)
optimization = GA_ABC(computation.obj_func_compute_energy, cluster_boundary, colony_size=10,  
                      max_iteration=0, initial_population_scaler=1, output_database = output_db_name )
optimization.run(print_interval=1)  # Start running. Print information with given frequency. 


In [ ]:
db = connect('structure_pool.db')
atoms = db.get_atoms(id= 3 ) 
view( atoms, viewer='x3d' )


In [ ]:
water = 'xyz_structures/Water.xyz'

input_molecules = [water]
input_num_of_molecules = [6]

input_constraint_type = [ 'micelle' ]
input_constraint_value = [ (3,4,5, (0,0,0), 1.8, 0,1) ]

# Setting cluster" 
cluster = cluster_model(input_molecules, input_num_of_molecules, input_constraint_type, input_constraint_value,)
cluster.init_molelcules()
cluster_template, cluster_boundary, cluster_conversion_rule = cluster.generate_bounds()  # Generate modeling setting

# With built-in coarse-optimization force field 
coarse_opt_parameter = dict(coarse_calc_eps='UFF', coarse_calc_sig='UFF', coarse_calc_chg=0, 
                            coarse_calc_step=20, coarse_calc_fmax=10, coarse_calc_constraint=None)
computation = energy_computation(templates = cluster_template, go_conversion_rule = cluster_conversion_rule,   
                                 calculator = None, geo_opt_para = None, calculator_type = 'structural',    
                                 if_coarse_calc = True, coarse_calc_para = coarse_opt_parameter,)
# Setting the main opt process
remove_db(output_db_name)
optimization = GA_ABC(computation.obj_func_compute_energy, cluster_boundary, colony_size=10,  
                      max_iteration=0, initial_population_scaler=1, output_database = output_db_name )
optimization.run(print_interval=1)  # Start running. Print information with given frequency. 

In [ ]:
db = connect('structure_pool.db')
atoms = db.get_atoms(id= 4 ) 
view( atoms, viewer='x3d' )


# Example 6: replace

In [ ]:
bulk_copper = 'xyz_structures/Bulk_copper.poscar' 
dopant = 'xyz_structures/Single_atom.xyz'

# What atoms can be replaced? The top layers in this case
atoms = read(bulk_copper)
top_layer = np.amax( atoms.positions[:,2] )
replace_atoms = [ at.index for at in atoms if at.position[2]>top_layer-0.1 ]

input_molecules = [bulk_copper, dopant]
input_num_of_molecules = [1, 6]

input_constraint_type = [ 'at_position', 'replace' ]
input_constraint_value = [ (), replace_atoms ]

# Setting cluster" 
cluster = cluster_model(input_molecules, input_num_of_molecules, input_constraint_type, input_constraint_value,
                       pbc_box=(14.31,14.31,14.31) ) # PBC system
cluster.init_molelcules()
cluster_template, cluster_boundary, cluster_conversion_rule = cluster.generate_bounds()  # Generate modeling setting

# Constraint
cluster_atoms = Atoms()
for at in cluster_template:
    cluster_atoms += at
ase_constraint = FixAtoms(indices=[at.index for at in cluster_atoms if at.symbol=='Cu' ])

# With built-in coarse-optimization force field 
coarse_opt_parameter = dict(coarse_calc_eps='UFF', coarse_calc_sig='UFF', coarse_calc_chg=0, 
                            coarse_calc_step=20, coarse_calc_fmax=10, coarse_calc_constraint=ase_constraint)
computation = energy_computation(templates = cluster_template, go_conversion_rule = cluster_conversion_rule,   
                                 calculator = None, geo_opt_para = None, calculator_type = 'structural',    
                                 if_coarse_calc = True, coarse_calc_para = coarse_opt_parameter,)
# Setting the main opt process
remove_db(output_db_name)
optimization = GA_ABC(computation.obj_func_compute_energy, cluster_boundary, colony_size=10,  
                      max_iteration=0, initial_population_scaler=1, output_database = output_db_name )
optimization.run(print_interval=1)  # Start running. Print information with given frequency. 

In [ ]:
db = connect('structure_pool.db')
atoms = db.get_atoms(id= 4 ) 
view( atoms, viewer='x3d' )
